In [1]:
# import the needed packages
import skmob
from skmob.tessellation.tilers import tiler
from skmob.utils.plot import plot_gdf

import pandas as pd
import geopandas as gpd

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi,voronoi_plot_2d
from geovoronoi import voronoi_regions_from_coords, points_to_coords
import random
from shapely.geometry import Polygon, Point
import folium

In [2]:
def get_convex_hull(tess):
    polygon = tess.at[0, 'geometry']
    for tile in tess['geometry']:
        polygon = polygon.union(tile)
    return polygon.convex_hull

# Defining the randomization generator
def polygon_random_points(poly, num_points):
    min_x, min_y, max_x, max_y = poly.bounds
    points = []
    while len(points) < num_points:
        random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            points.append([random_point.x, random_point.y])
    return np.array(points)

def to_GeoDataFrame(region_polys):
    name=[]
    for i in range(1, len(region_polys) + 1):
        name.append('cell ' + str(i))
    gdf = gpd.GeoDataFrame(columns=['name','geometry'], crs={'init': 'epsg:4326'})
    gdf['name'] = name
    for index, row in gdf.iterrows():
        gdf.at[index, 'geometry'] = region_polys[index]
    return gdf

def get_voronoi_tessellation(poly_ch, points):
    vor = Voronoi(points, qhull_options='Qbb Qc Qx')
    #fig = voronoi_plot_2d(vor)
    #plt.show()
    region_polys, region_pts = voronoi_regions_from_coords(points, poly_ch)
    tess_voronoi = to_GeoDataFrame(region_polys)
    return tess_voronoi

In [4]:
# Creating the squared tesselation, with tiles of 500m for the city of Florence
tess_squared = tiler.get('squared', base_shape='Italy', meters=500)
print("tiles = %s" %len(tess_squared))
tess_squared.head()

ConnectionError: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search.php?q=Italy&polygon_geojson=1&format=json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000144A59A32B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [5]:
poly_ch = get_convex_hull(tess_squared)
# Choosing the number of points desired 
points = polygon_random_points(poly_ch, 100)

NameError: name 'tess_squared' is not defined